In [1]:
#Ignore scikit learn deprication warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re


/Users/andrew/workspace/endangeredanimals/learning


In [3]:
from numpy import genfromtxt
# train = pd.read_csv('train_reduced.csv', encoding="ISO-8859-1", index_col='_id')
# test = pd.read_csv('test_reduced.csv', encoding="ISO-8859-1", index_col='_id')
from numpy import genfromtxt
#myData = genfromtxt('train.csv',delimiter=',')

#train = genfromtxt('train_reduced_CME_test.csv',delimiter=',')
#test = genfromtxt('test_reduced_CME_test.csv',delimiter=',')

train = genfromtxt('train_reduced.csv',delimiter=',')
test = genfromtxt('test_reduced.csv',delimiter=',')

print(train.shape)
# test = test.dropna()
# test.target
train = train[~np.isnan(train).any(axis=1)]

y = train[:,4] # label
X = train[:,0:3] # data

(280, 9)


In [4]:
# from sklearn.cross_validation import cross_val_score

# # TODO use F-1 for scoring, or a hybrid if possible
# # we can use a for loop ['f1', 'accuracy', ]
# # http://scikit-learn.org/stable/modules/model_evaluation.html

# # logistic regression
# from sklearn.linear_model import LogisticRegression
# logreg = LogisticRegression()
# logreg.fit(X,y)
# print('Logistic Regression:')
# print(, cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

# # decision tree
# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier()
# clf.fit(X,y)
# print('Decision Tree:', cross_val_score(clf, X, y, cv=10, scoring='accuracy').mean())

# # random forest
# from sklearn.ensemble import RandomForestClassifier
# forest = RandomForestClassifier(n_estimators=10, max_depth=10)
# forest.fit(X,y)
# print('Random Forest:', cross_val_score(forest, X, y, cv=10, scoring='accuracy').mean())

# # support vector machine
# from sklearn.svm import LinearSVC
# svm = LinearSVC()
# svm.fit(X,y)
# print('SVM:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# # naive bayes
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# gnb.fit(X,y)
# print('Naive Bayes:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# # KNN
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=20)
# knn.fit(X,y)
# print('KNN:', cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())


In [5]:
train = genfromtxt('train_reduced.csv',delimiter=',')


In [27]:
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('train_reduced.csv',delimiter=',')
print(train.shape)
print(train[6,:])
print
print(train[6,train.shape[1]-1])
len(train)

# get rid of first row (I think this is just column labels)
train = train[0:,:]

# replace missing featureVals with 0.5 (halfway in between)
where_are_NaNs = np.isnan(train)
train[where_are_NaNs] = -1

y = train[:,train.shape[1]-1] # label
X = train[:,0:train.shape[1]-2] # data
print('y')
print(y.shape)
print(y[0:5])
print('X')
print(X.shape)
print(X[0:5])

(280, 9)
[  4.22815800e+06   1.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   1.45454545e-01   0.00000000e+00   4.54545455e-02
   1.00000000e+00]
1.0
y
(280,)
[-1.  0.  0.  0.  0.]
X
(280, 7)
[[ -1.00000000e+00  -1.00000000e+00  -1.00000000e+00  -1.00000000e+00
   -1.00000000e+00  -1.00000000e+00  -1.00000000e+00]
 [  7.19449900e+06   1.00000000e+00   1.00000000e+00   0.00000000e+00
    3.33333333e-01   1.33333333e-01   0.00000000e+00]
 [  2.11174300e+06   1.00000000e+00   0.00000000e+00   0.00000000e+00
    6.00000000e-01   6.53846154e-01   0.00000000e+00]
 [  5.82845500e+06   1.00000000e+00   0.00000000e+00   0.00000000e+00
   -1.00000000e+00  -1.00000000e+00   0.00000000e+00]
 [  7.69291500e+06   0.00000000e+00   1.00000000e+00   0.00000000e+00
   -1.00000000e+00  -1.00000000e+00   0.00000000e+00]]


###### 

In [28]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

models = [LogisticRegression(), 
          DecisionTreeClassifier(), 
          RandomForestClassifier(n_estimators=10, max_depth=10),
          LinearSVC(),
          GaussianNB(),
          KNeighborsClassifier(n_neighbors=20)
         ]
names =  ['Logistic Regression', 
          'Decision Tree',
          'Random Forest',
          'SVM',
          'Naive Bayes',
          'KNN']

metrics = ['accuracy', 'precision', 'recall', 'f1']

def compute_metrics(name, model):
    model.fit(X,y)
    print("{}:".format(name))
    for metric in metrics:
        print(metric + ':', cross_val_score(model, X, y, cv=10, scoring=metric).mean())
    print('')

for i in range(len(names)):
    compute_metrics(names[i], models[i])

Logistic Regression:
accuracy: 0.861111111111


ValueError: Target is multiclass but average='binary'. Please choose another average setting.